In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage.io
from tqdm.notebook import tqdm
import random
import cv2
import seaborn as sns
import os
# plt.rcParams['figure.figsize'] = [15,8]
from __future__ import print_function, division
import torch
import os
import torch
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from tqdm import tqdm

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode


Have referred the code for data analysis and usage from some notebooks in the competition thread.

In [ ]:
# !pip install tqdm
# # 

In [ ]:
DATA_DIR = './'
train_csv = pd.read_csv(DATA_DIR + 'train.csv')
train_csv.head()


In [ ]:
train_csv.describe()

In [ ]:
train_csv['sex'].unique()
# train_csv['sex'].nunique()
train_csv.sex.value_counts()

In [ ]:
train_csv.age_approx.value_counts()

In [ ]:
train_csv.groupby(['target']).size().plot(kind='bar')

In [ ]:
train_csv.anatom_site_general_challenge.unique()
# train_csv.anatom_site_general_challenge.value_counts()
train_csv['anatom_site_general_challenge'].isnull().value_counts
#total 32577

In [ ]:
train_csv.isnull().sum()

In [ ]:
train_csv.target.value_counts()

In [ ]:
train_csv.benign_malignant.value_counts()

In [ ]:
N=33126
random_index = random.randint(0,N)
# print(train_csv.iloc[random_index])
#train_csv.iloc[random_index]['image_name'] 
img = DATA_DIR + 'data/train/' + train_csv.iloc[random_index]['image_name']  + '.jpg'
# img = DATA_DIR + 'jpeg/train/' +  'ISIC_0272509' + '.jpg'
try:
    img = DATA_DIR + 'data/train/' + train_csv.iloc[random_index]['image_name']  + '.jpg'
    plt.imshow(mpimg.imread(img))
except:
    print('in except')
    img = DATA_DIR + 'data/val/' + train_csv.iloc[random_index]['image_name']  + '.jpg'
    plt.imshow(mpimg.imread(img))

In [ ]:
%%html
Links to understand if pretrained imagenet model should be used or a smaller CNN :
https://ai.googleblog.com/2019/12/understanding-transfer-learning-for.html
http://openaccess.thecvf.com/content_ECCVW_2018/papers/11134/Xie_Pre-training_on_Grayscale_ImageNet_Improves_Medical_Image_Classification_ECCVW_2018_paper.pdf

Converting images to grayscale images for the given dataset seems like a good idea since people of different skin types are likely to be present.

In [ ]:
#df['Charge_Per_Line'] = df['Charge_Per_Line'].replace('-', np.nan)
train_csv = train_csv.dropna(axis=0, subset=['sex'])
train_csv = train_csv.dropna(axis=0, subset=['age_approx'])

In [ ]:
train_csv.isnull().sum()

In [ ]:
train_csv.groupby(['anatom_site_general_challenge']).size().plot(kind='bar')

In [ ]:
train_csv = train_csv.dropna(axis=0, subset=['anatom_site_general_challenge'])

In [ ]:
train_csv.groupby(['diagnosis']).size().plot(kind='bar')

In [ ]:
del train_csv['diagnosis']


In [ ]:
del train_csv['patient_id']
del train_csv['benign_malignant']

In [ ]:
encoder = {"sex":     {"male": 0, "female": 1},
            "benign_malignant": {"benign": 0, "malignant": 1 },
 "anatom_site_general_challenge": {"head/neck":0, "upper extremity":1, "lower extremity":2, "torso":3, "palms/soles":4, "oral/genital":5}}
train_csv.replace(encoder, inplace=True)
train_csv.head()

In [ ]:
train_csv[1:100]

In [ ]:
train_csv['anatom_site_general_challenge'].unique()

In [ ]:
train_csv.target.value_counts()

In [ ]:
is_infected = train_csv['target'] == 1
df_try = train_csv[is_infected]
print(len(df_try))
train_csv=train_csv.append([df_try]*53,ignore_index=True)
print(len(train_csv))

In [ ]:
root_dir='./data/'
train_csv['directory'] = -1
print(train_csv.shape[0])
train_benign = 0
train_malignant = 0
val_benign = 0
val_malignant=0
for idx in range(0, train_csv.shape[0]):
    try:
        img_name = os.path.join(root_dir, 'train',
                                train_csv.iloc[idx, 0]) + '.jpg'
        image = io.imread(img_name)
        train_csv.iloc[idx, train_csv.columns.get_loc('directory')] = 0
        if train_csv.iloc[idx, train_csv.columns.get_loc('target')] == 0 :
            train_benign += 1 
        else:
            train_malignant += 1
    except:
        img_name = os.path.join(root_dir, 'val',
                                train_csv.iloc[idx, 0]) + '.jpg'
        image = io.imread(img_name)
        train_csv.iloc[idx, train_csv.columns.get_loc('directory')] = 1
        if train_csv.iloc[idx, train_csv.columns.get_loc('target')] == 0 :
            val_benign += 1 
        else:
            val_malignant += 1
print(train_benign)
print(train_malignant)
print(val_benign)
print(val_malignant)

In [ ]:
total_train = train_benign + train_malignant
benign_weight = train_benign/total_train
malignant_weight = train_malignant/total_train
# print('train: ', train)
# print('val: ', val)
benign_weight = 0.98
malignant_weight = 0.02
print('benign_weight: ', benign_weight)
print('malignant_weight: ', malignant_weight)
train_csv.directory.value_counts()

In [ ]:
val_csv = train_csv[train_csv.directory == 1]
train_csv = train_csv[train_csv.directory == 0]


In [ ]:
train_csv.directory.value_counts()

In [ ]:
class MelanomaCancerDataset(Dataset):
    """Melanoma Cancer dataset."""

    def __init__(self, csv_dataframe, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv_dataframe = csv_dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.csv_dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.csv_dataframe.iloc[idx, 0]) + '.jpg'
        image_name = self.csv_dataframe.iloc[idx, 0]
        image = io.imread(img_name)
        csv_data = self.csv_dataframe.iloc[idx, 1:4]
        csv_data = np.array([csv_data])
        csv_data = csv_data.astype('double').reshape(-1, 3).flatten()
        labels = self.csv_dataframe.iloc[idx, 4]
        labels = labels.reshape(-1, 1).flatten()
        sample = {'image': image, 'csv_data': csv_data, 'labels': labels, 'image_name': image_name }

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
melanoma_dataset = MelanomaCancerDataset(csv_dataframe=train_csv,
                                    root_dir='./data/train/')

fig = plt.figure()

for i in range(len(melanoma_dataset)):
    sample = melanoma_dataset[i]

    print(i, sample['image'].shape, sample['csv_data'].shape, sample['labels'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    plt.imshow(sample['image'])

    if i == 3:
        plt.show()
        break

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, csv_data, labels = sample['image'], sample['csv_data'], sample['labels']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
#         landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'csv_data': csv_data, 'labels': labels}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, csv_data, labels = sample['image'], sample['csv_data'], sample['labels']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

#         landmarks = landmarks - [left, top]

        return {'image': image, 'csv_data': csv_data, 'labels': labels}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, csv_data, labels, image_name = sample['image'], sample['csv_data'], sample['labels'], sample['image_name']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'csv_data': torch.from_numpy(csv_data), 'labels': torch.from_numpy(labels), 'image_name': image_name }

In [ ]:

# crop = RandomCrop(1000)
# scale = Rescale(128)
# composed = transforms.Compose([Rescale(1024),
#                                RandomCrop(128)])

# # Apply each of the above transforms on sample.
# fig = plt.figure()
# sample = melanoma_dataset[65]
# for i, tsfrm in enumerate([scale, crop, composed]):
#     transformed_sample = tsfrm(sample)

#     ax = plt.subplot(1, 3, i + 1)
#     plt.tight_layout()
#     ax.set_title(type(tsfrm).__name__)
#     plt.imshow(sample['image'])

# plt.show()

In [ ]:
transformed_dataset = MelanomaCancerDataset(csv_dataframe=train_csv,
                                            root_dir='./data/train/',
#                                              transform=transforms.Compose([
#                                                 transforms.Resize(64),  # resize the image to 64x64 (remove if images are already 64x64)
#                                                 transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
#                                                 ToTensor()]))
                                           transform=transforms.Compose([
#                                                Rescale(256),
#                                                RandomCrop(64),
                                               ToTensor()
                                           ]))
val_transformed_dataset = MelanomaCancerDataset(csv_dataframe=val_csv,
                                            root_dir='./data/val/',
#                                              transform=transforms.Compose([
#                                                 transforms.Resize(64),  # resize the image to 64x64 (remove if images are already 64x64)
#                                                 transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
#                                                 ToTensor()]))
                                           transform=transforms.Compose([
#                                                Rescale(256),
#                                                RandomCrop(64),
                                               ToTensor()
                                           ]))
# print(transformed_dataset[0])
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['image'].size(), sample['csv_data'].size(), sample['labels'].size())

    if i == 3:
        break

In [ ]:
dataloader = DataLoader(transformed_dataset, batch_size=512,
                        shuffle=True, pin_memory=True)
val_dataloader = DataLoader(val_transformed_dataset, batch_size=512,
                        shuffle=True, pin_memory=True)
## 512 batch size takes long time
##1024 batch size mem not enough
#, num_workers=4


# Helper function to show a batch
def show_batch(sample_batched):
    """Show image for a batch of samples."""
    images_batch, csv_data = \
            sample_batched['image'], sample_batched['csv_data']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid_border_size = 2

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    for i in range(batch_size):
        print('heree')
#         plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size + (i + 1) * grid_border_size,
#                     landmarks_batch[i, :, 1].numpy() + grid_border_size,
#                     s=10, marker='.', c='r')

#         plt.title('Batch from dataloader')

# for i_batch, sample_batched in enumerate(dataloader):
#     print(i_batch, sample_batched['image'].size(),
#           sample_batched['csv_data'].size())

#     # observe 4th batch and stop.
#     if i_batch == 3:
#         plt.figure()
#         show_batch(sample_batched)
#         plt.axis('off')
#         plt.ioff()
#         plt.show()
#         break

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class imageModel(nn.Module):

    def __init__(self):
        super(imageModel, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        self.pool1 = nn.MaxPool2d(3,1)
        self.pool2 = nn.MaxPool2d(3,3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(2048, 64)  # 6*6 from image dimension
#         self.fc2 = nn.Linear(120, 64)
#         self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
#         print(x.shape)
        x = self.pool1(F.relu(self.conv1(x)))
#         print(x.shape)
        # If the size is a square you can only specify a single number
        x = self.pool2(F.relu(self.conv2(x)))
#         print(x.shape)
        x = self.pool2(F.relu(self.conv3(x)))
#         print(x.shape)
        x = x.view(-1, self.num_flat_features(x))
#         print('flat features')
#         print(x.shape)
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.sigmoid(x)
        return x

    def num_flat_features(self, x):
#         return 32768
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


imageModel = imageModel()
print(imageModel)
class melanomaModel(nn.Module):
    def __init__(self):
        super(melanomaModel, self).__init__()
        self.cnn =imageModel
#         self.cnn.fc = nn.Linear(
#             self.cnn.fc.in_features, 20)
        self.pi = 0.01
        self.bias_init = np.log10((1-self.pi)/self.pi)
        print('self.bias_init: ', self.bias_init)
        self.fc1 = nn.Linear(64 + 3, 32)
        self.fc2 = nn.Linear(32, 1)
#         self.fc2.bias.data.fill_(self.bias_init)
#         self.fc2 = nn.Linear(60, 5)
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        x2 = data
#         print('x1.shape')
#         print(x1.shape)
#         print('x2.shape')
#         print(x2.shape)
        
        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
#         x = F.sigmoid(x)
#         print(x.shape)
        return x
        

# model = melanomaModel()

# batch_size = 2
# image = torch.randn(batch_size, 3, 299, 299)
# data = torch.randn(batch_size, 10)

# output = model(image, data)

In [ ]:
malignant_weight = 0.5
benign_weight = 0.95
pos_weight = torch.tensor([494])
def weighted_binary_cross_entropy(output, target, weights=torch.tensor([malignant_weight, benign_weight])):
        
    if weights is not None:
        assert len(weights) == 2
        
        loss = weights[1] * (target * torch.log(output)) + weights[0] * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))

def focal_loss(inputs, targets, alpha=0.8, gamma=2, logits=False, reduce=True):
#     print('------------- inside loss')
#     print(inputs.shape)
#     print(inputs)
    if logits:
#         print('------heree in if')
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
    else:
#         print('------heree in elseeeeeeeeeeeeeeeeeeeeeee')
        BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
#     print('BCELOSS', BCE_loss)
    pt = torch.exp(-BCE_loss)
#     print('pt', pt)
    F_loss = alpha * (1-pt)**gamma * BCE_loss
#     print(F_loss)

    if reduce:
        return torch.mean(F_loss)
    else:
        return F_loss
    

class RunningAverage():
    """A simple class that maintains the running average of a quantity
    
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """
    def __init__(self):
        self.steps = 0
        self.total = 0
    
    def update(self, val):
        self.total += val
        self.steps += 1
    
    def __call__(self):
        return self.total/float(self.steps)

def train(model, optimizer, dataloader, metrics, params):
    """Train the model on `num_steps` batches

    Args:
        model: (torch.nn.Module) the neural network
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        num_steps: (int) number of batches to train on, each of size params.batch_size
    """

    # set model to training mode
    model.train()

    # summary for current training loop and a running average object for loss
    summ = []
    loss_avg = RunningAverage()
#     criterion = weighted_binary_cross_entropy
    criterion = nn.BCEWithLogitsLoss()

    # Use tqdm for progress bar
    with tqdm(total=len(dataloader)) as t:
        for i, sample_batched in enumerate(dataloader):
            # move to GPU if available
            image, csv_data, labels_batch = sample_batched['image'].float(), sample_batched['csv_data'].float(), sample_batched['labels'].float()
            if params.cuda:
                image, csv_data, labels_batch = image.cuda(non_blocking=True), csv_data.cuda(non_blocking=True), labels_batch.cuda(non_blocking=True)
            # convert to torch Variables
#             image, csv_data, labels = Variable(
#                 image), Variable(csv_data), Variable(labels_batch)

            # compute model output and loss
            output_batch = model(image, csv_data)
            loss = criterion(output_batch, labels_batch) 

            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            loss.backward()

            # performs updates using calculated gradients
            optimizer.step()

            # Evaluate summaries only once in a while
            if i % params.save_summary_steps == 0:
                # extract data from torch Variable, move to cpu, convert to numpy arrays
                output_batch = output_batch.data.cpu().numpy()
                labels_batch = labels_batch.data.cpu().numpy()

                # compute all metrics on this batch
                summary_batch = {metric: metrics[metric](output_batch, labels_batch)
                                 for metric in metrics}
                summary_batch['loss'] = loss.item()
                summ.append(summary_batch)

            # update the average loss
            loss_avg.update(loss.item())

            t.set_postfix(loss='{:05.3f}'.format(loss_avg()))
            t.update()

    # compute mean of all metrics in summary
    metrics_mean = {metric: np.mean([x[metric]
                                     for x in summ]) for metric in summ[0]}
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v)
                                for k, v in metrics_mean.items())
    print("- Train metrics: " + metrics_string)


In [ ]:
def save_dict_to_json(d, json_path):
    """Saves dict of floats in json file

    Args:
        d: (dict) of float-castable values (np.float, int, float, etc.)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        # We need to convert the values to float for json (it doesn't accept np.array, np.float, )
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)

def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer, metrics, params, model_dir,
                       restore_file=None):
    """Train the model and evaluate every epoch.

    Args:
        model: (torch.nn.Module) the neural network
        train_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches training data
        val_dataloader: (DataLoader) a torch.utils.data.DataLoader object that fetches validation data
        optimizer: (torch.optim) optimizer for parameters of model
        loss_fn: a function that takes batch_output and batch_labels and computes the loss for the batch
        metrics: (dict) a dictionary of functions that compute a metric using the output and labels of each batch
        params: (Params) hyperparameters
        model_dir: (string) directory containing config, weights and log
        restore_file: (string) optional- name of file to restore from (without its extension .pth.tar)
    """
    # reload weights from restore_file if specified
    if restore_file is not None:
        print('restored file')
        checkpoint = torch.load(params.PATH)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#         restore_path = os.path.join(
#             args.model_dir, args.restore_file + '.pth.tar')
#         logging.info("Restoring parameters from {}".format(restore_path))
#         utils.load_checkpoint(restore_path, model, optimizer)

    best_val_acc = 0.0

    for epoch in range(params.num_epochs):
        # Run one epoch
        print("Epoch {}/{}".format(epoch + 1, params.num_epochs))

        # compute number of batches in one epoch (one full pass over the training set)
        train(model, optimizer, train_dataloader, metrics, params)

        # Evaluate for one epoch on validation set
#         val_metrics = evaluate(model, val_dataloader, metrics, params)

#         val_acc = val_metrics['accuracy']
#         is_best = val_acc >= best_val_acc

        # Save weights
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
#             'loss': loss
#             is_best=is_best
            }, params.PATH)
#         utils.save_checkpoint({'epoch': epoch + 1,
#                                'state_dict': model.state_dict(),
#                                'optim_dict': optimizer.state_dict()},
#                               is_best=is_best,
#                               checkpoint=model_dir)

        # If best_eval, best_save_path
#         if is_best:
#             logging.info("- Found new best accuracy")
#             best_val_acc = val_acc

#             # Save best val metrics in a json file in the model directory
#             best_json_path = os.path.join(
#                 model_dir, "metrics_val_best_weights.json")
#             save_dict_to_json(val_metrics, best_json_path)

#         # Save latest val metrics in a json file in the model directory
#         last_json_path = os.path.join(
#             model_dir, "metrics_val_last_weights.json")
#         save_dict_to_json(val_metrics, last_json_path)

In [ ]:

import torch.optim as optim
import gc
def accuracy(outputs, labels):
    """
    Compute the accuracy, given the outputs and labels for all images.

    Args:
        outputs: (np.ndarray) dimension batch_size x 6 - log softmax output of the model
        labels: (np.ndarray) dimension batch_size, where each element is a value in [0, 1, 2, 3, 4, 5]

    Returns: (float) accuracy in [0,1]
    """
    outputs = np.argmax(outputs, axis=1)
    return np.sum(outputs==labels)/float(labels.size)

def binary_acc(y_pred, y_test):
    y_pred_tag = np.round(y_pred)
#     y_pred_tag=y_pred
#     print(y_pred)
    print('------------------')
#     print(y_pred_tag)

    correct_results_sum = float((y_pred_tag == y_test).sum())
    actual_one_results_sum = (y_test==1).sum()
    actual_zero_results_sum = (y_test==0).sum()
    predicted_one_results_sum = (y_pred_tag == 1).sum()
    predicted_zero_results_sum = (y_pred_tag == 0).sum()
    acc = correct_results_sum/y_test.shape[0]
    acc = np.round(acc * 100)
    print('correct_results_sum: ', correct_results_sum)
    print('actual_one_results_sum: ', actual_one_results_sum)
    print('actual_zero_results_sum: ', actual_zero_results_sum)
    print('predicted_one_results_sum: ', predicted_one_results_sum)
    print('predicted_zero_results_sum: ', predicted_zero_results_sum)
    print('y_test.shape[0]: ', y_test.shape[0])
#     acc=-1
    
    return acc

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

metrics = {
    'accuracy': binary_acc,
    # could add more metrics such as accuracy for each token type
}
params = {
    "learning_rate": 1e-3,
    "batch_size": 128,
    "num_epochs": 150,
    "dropout_rate":0.8, 
    "num_channels": 32,
    "save_summary_steps": 200,
    "num_workers": 4,
    "PATH": r'\trained_weights\weights.pth'
}
params['cuda'] = torch.cuda.is_available()
params = dotdict(params)
print(params.learning_rate)
model = melanomaModel().cuda() if params.cuda else melanomaModel()
# optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
# optimizer = optim.SGD(model.parameters(), params.learning_rate)
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.5)
train_and_evaluate(model, dataloader, val_dataloader, optimizer, metrics, params, 'ML\\Melanoma Classification\\',
                       'yes')

In [ ]:
test_csv = pd.read_csv(DATA_DIR + 'test.csv')
test_csv.head()
print(test_csv.shape)
test_csv.anatom_site_general_challenge.value_counts()
del test_csv['patient_id']
test_csv["anatom_site_general_challenge"].fillna("na", inplace = True) 
test_csv.isnull().sum()
encoder = {"sex":     {"male": 0, "female": 1},
 "anatom_site_general_challenge": {"head/neck":0, "upper extremity":1, "lower extremity":2, "torso":3, "palms/soles":4, "oral/genital":5, "na":6}}
test_csv.replace(encoder, inplace=True)

test_csv.isnull().sum()
test_csv['target'] = -1



In [ ]:
test_csv.head()

In [ ]:
test_csv.anatom_site_general_challenge.value_counts()
transformed_test_dataset = MelanomaCancerDataset(csv_dataframe=test_csv,
                                            root_dir='./data/test/',
#                                              transform=transforms.Compose([
#                                                 transforms.Resize(64),  # resize the image to 64x64 (remove if images are already 64x64)
#                                                 transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
#                                                 ToTensor()]))
                                           transform=transforms.Compose([
#                                                Rescale(256),
#                                                RandomCrop(64),
                                               ToTensor()
                                           ]))

In [ ]:
dataloader = DataLoader(transformed_test_dataset,batch_size=32)

PATH = r'\Melanoma Classification\trained_weights\weights.pth'
checkpoint = torch.load(PATH)
model = melanomaModel().cuda() 
model.load_state_dict(checkpoint['model_state_dict'])
values = []
for i, sample_batched in enumerate(dataloader):
#     print(sample_batched)
    image, csv_data, labels_batch, image_names = sample_batched['image'].float(), sample_batched['csv_data'].float(), sample_batched['labels'].float(), sample_batched['image_name']
    if params.cuda:
        image, csv_data, labels_batch = image.cuda(non_blocking=True), csv_data.cuda(non_blocking=True), labels_batch.cuda(non_blocking=True)
    outputs = model(image, csv_data)
    outputs = outputs.data.cpu().numpy()
    t=[]
    t.append(image_names)
    image_names = np.asarray(t).transpose()
#     print(image_names.shape)
#     print(outputs.shape)
    final_values = np.concatenate([image_names, outputs], axis=1)
    for j in range(final_values.shape[0]):
        values.append(final_values[j].tolist())
sample_submission = pd.DataFrame(values, columns=['image_name', 'target'])
sample_submission.to_csv('submission.csv', index=False)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
print(sample_submission)

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)
model = melanomaModel()
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
#  IMG_HEIGHT = 350
# IMG_WIDTH = 300
# IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
# base_model = tf.keras.applications.VGG19(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=(1024, 1024, 3)
# )
# base_model.trainable = False
# for layer in base_model.layers:
#     layer.trainable = False
#     if isinstance(layer, tf.keras.layers.BatchNormalization):
#         layer._per_input_updates = {}

# image_model = models.Sequential()
# image_model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
# image_model.add(layers.MaxPooling2D((3, 3)))
# image_model.add(layers.Conv2D(64, (5, 5), activation='relu'))
# image_model.add(layers.MaxPooling2D((3, 3)))
# image_model.add(layers.Conv2D(128, (5, 5), activation='relu'))
# image_model.add(layers.MaxPooling2D((3, 3)))
# image_model.add(layers.Flatten())
# image_model.add(layers.Dense(64, activation='relu'))

In [ ]:
print("torch.cuda.is_available() =", torch.cuda.is_available())
print("torch.cuda.device_count() =", torch.cuda.device_count())
print("torch.cuda.device('cuda') =", torch.cuda.device('cuda'))
print("torch.cuda.current_device() =", torch.cuda.current_device())
print("torch.cuda.get_device_name(0) =",torch.cuda.get_device_name(0))
print("torch.cuda.get_device_name() =",torch.cuda.get_device_name())
print(torch.cuda.device(0))
# cudnn.benchmark = True
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:',torch.cuda.memory_allocated(0))
    print('Cached: ', round(torch.cuda.memory_cached(0)))

In [ ]:
# pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

#### layer = tf.keras.layers.Flatten()
# dense_layer = tf.keras.layers.Dense(1024, activation='relu')
# prediction_layer = tf.keras.layers.Dense(8, activation='softmax')
# model = tf.keras.Sequential([
#   base_model,
#   layer,
#   dense_layer,
#   prediction_layer
# ])
# base_learning_rate = 0.0001
# model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
#               loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
#               metrics=['accuracy'])
# model.summary()

In [ ]:
!python --version

In [ ]:

+++++-